<a href="https://colab.research.google.com/github/Romqa41/GPN/blob/main/%D0%AD%D0%94%D0%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок импорта библиотек

In [33]:
import pandas as pd
import numpy as np
import re

# Блок загрузки данных

In [34]:
# приход
inc = pd.read_excel('Приход.XLSX', dtype={
    'Поставщик': 'Int64',
    'Отпускающий завод':'Int64'
    })

# расход
exp = pd.read_excel('Расход.XLSX')

# зеркало мх1
mirr_mx1 = pd.read_excel('зерк мх1.XLSX', dtype={
    'ВДв матер. ГПН-Снаб.':'Int64',
    'ГодДокумМатериала.1':'Int64',
    'ГодДокумМатериала.2':'Int64'
    })

# зеркало мх3
mirr_mx3 = pd.read_excel('зерк мх3.XLSX', dtype={
    'ВДв матер. ГПН-Снаб.':'Int64',
    'ГодДокумМатериала.1':'Int64',
    'ГодДокумМатериала.2':'Int64'
    })

# ZINMM521 после зерк мх1 и мх2
zinmm521_mirr_mx1 = pd.read_excel('521 после зерк мх1.XLSX')
zinmm521_mirr_mx3 = pd.read_excel('521 после зерк мх3.XLSX')

# справочник
manual = pd.read_excel('Справочник ЭДО.xlsx')

# свод
final = pd.read_excel('Для свода.xlsx')

# Блок рабочего кода

## Приход

In [35]:
# для всех заводов и складов фиксируем формат - четыре цифры, по необходимости дополнеяем ведущими нулями
inc['Принимающий завод'] = inc['Принимающий завод'].astype(str).str.zfill(4).str.upper()
inc['Принимающий склад'] = inc['Принимающий склад'].astype(str).str.zfill(4).str.upper()
inc['Отпускающий завод'] = inc['Отпускающий завод'].astype(str).str.zfill(4).str.upper()
inc['Отпускающий склад'] = inc['Отпускающий склад'].astype(str).str.zfill(4).str.upper()

# исключаем фактор регистра (приводим все значения к верхнему)
manual['сцепка на мх1/м11'] = manual['сцепка на мх1/м11'].str.upper()

In [36]:
# получаем столбец "Документ материала ГПН-Снаб"
inc = inc.merge(
    mirr_mx1[['Документ материала ДЗО', 'Документ материала ГПН-Снаб']],
    left_on='Документ материала',
    right_on='Документ материала ДЗО',
    how='left'
    ).drop('Документ материала ДЗО', axis=1)

# получаем столбец "Сцепка"
inc['Сцепка'] = inc['Принимающий завод'] + inc['Принимающий склад']

# получаем столбец "Исключение перемещения"
inc['Исключение перемещения'] = np.where(
    (inc['Принимающий склад']+inc['Принимающий завод'])==(inc['Отпускающий склад']+inc['Отпускающий завод']), 0, 1
    )

# создаем словарь с количеством вхождений значений столбца "сцепка на мх1/м11" в справочнике
counts = manual['сцепка на мх1/м11'].value_counts().to_dict()

# получаем столбец "Количество М4"
inc['Количество М4'] = inc['Сцепка'].map(counts).fillna(0).astype(int) * inc['Исключение перемещения']

# получаем столбец "Электронный ПУД М4"
inc['Электронный ПУД М4'] = np.where(inc['НЭД'].isna(), 0, 1) * inc['Количество М4']

# получаем столбец "Дивизион"
inc = (inc
       .merge(manual[['сцепка на мх1/м11', 'Дивизион.1']], left_on='Сцепка', right_on='сцепка на мх1/м11', how='left')
       .drop('сцепка на мх1/м11', axis=1)
       .rename(columns={'Дивизион.1':'Дивизион'})
)

# получаем столбец "МХ1"
inc['МХ1'] = np.where(inc['Документ материала ГПН-Снаб'].isna(), 0, 1) * inc['Количество М4']

# получаем столбец "МХ1 НЭД"
inc = (inc
       .merge(
           zinmm521_mirr_mx1[['Документ материала', 'НЭД']],
           left_on='Документ материала ГПН-Снаб',
           right_on='Документ материала',
           how='left'
           )
       .drop('Документ материала_y', axis=1)
       .rename(columns={'Документ материала_x':'Документ материала', 'НЭД_x':'НЭД', 'НЭД_y':'МХ1 НЭД'})
)

# получаем столбец "Электронный ПУД МХ1"
inc['Электронный ПУД МХ1'] = np.where(inc['МХ1 НЭД'].isna(), 0, 1) * inc['МХ1']

In [38]:
# обработка пропусков (заполнение '' в колонках типа float)
for col in inc[['Документ материала ГПН-Снаб', 'Заказ на поставку', 'НЭД', 'МХ1 НЭД', 'Поставщик']]:
  inc[col] = inc[col].astype('Int64').fillna(0).astype(str)
  inc.loc[inc[col] == '0', col] = ''

# обработка пропусков (заполнение '' в колонках типа str)
for col in inc[[
    'Транспортная накладная', 'Вид докум. закупки', 'Принимающий склад', 'Принимающий завод',
    'Отпускающий склад', 'Отпускающий завод', 'Дивизион'
    ]]:
  inc[col] = inc[col].fillna('').replace('<NA>', '')

# отдельно обработка поля "Отпускающий склад"
inc['Отпускающий склад'] = np.where(inc['Отпускающий склад'].str.endswith('NAN'), '', inc['Отпускающий склад'])

In [39]:
# пересортировка столбцов (воспроизведение порядка как в Excel)
inc = inc[[
    'Поставщик', 'Документ материала ГПН-Снаб', 'Документ материала', 'Вид движения', 'Заказ на поставку',
    'Транспортная накладная', 'Вид докум. закупки', 'Сцепка', 'Количество М4', 'Принимающий склад',
    'Отпускающий склад', 'Принимающий завод', 'Отпускающий завод', 'НЭД', 'Электронный ПУД М4', 'Дивизион',
    'Класс оценки', 'МХ1', 'МХ1 НЭД', 'Электронный ПУД МХ1', 'Исключение перемещения'
]]

## Расход

In [40]:
# дополняем нулями при необходимости, заменяем пропуски на ''
exp['Принимающий завод'] = exp['Принимающий завод'].astype('Int64').astype(str).str.zfill(4).str.upper().str.replace('<NA>', '')
exp['Отпускающий завод'] = exp['Отпускающий завод'].astype('Int64').astype(str).str.zfill(4).str.upper().str.replace('<NA>', '')

# дополняем нулями при необходимости поле "Класс оценки"
exp['Класс оценки'] = exp['Класс оценки'].astype(str).str.zfill(4).str.upper()

# заменяем пропуски на ''
exp['Принимающий склад'] = exp['Принимающий склад'].fillna('')
exp['Отпускающий склад'] = exp['Отпускающий склад'].fillna('')

# заменяем пропуски на ''
exp['Принимающий склад наименование'] = exp['Принимающий склад наименование'].fillna('')

# заменяем пропуски на ''
exp['Вид докум. закупки'] = exp['Вид докум. закупки'].fillna('')

# заменяем пропуски на ''
exp['НЭД'] = exp['НЭД'].astype('Int64').fillna(0).astype(str)
exp.loc[exp['НЭД'] == '0', 'НЭД'] = ''

# заменяем тип данных на строковый
zinmm521_mirr_mx3['Документ материала'] = zinmm521_mirr_mx3['Документ материала'].astype(str)

# заменяем пропуски на ''
zinmm521_mirr_mx3['НЭД'] = zinmm521_mirr_mx3['НЭД'].astype('Int64').fillna(0).astype(str)
zinmm521_mirr_mx3.loc[zinmm521_mirr_mx3['НЭД'] == '0', 'НЭД'] = ''

In [41]:
# получаем столбец "Сцепка МХ-3"
exp['Сцепка МХ-3'] = exp['Отпускающий завод'] + exp['Принимающий завод'] + exp['Отпускающий склад']

# создаем словарь с количеством вхождений значений столбца "сцепка на мх3" в справочнике
counts_ = manual['сцепка на мх3'].value_counts().to_dict()

# получаем столбец "Движение требует МХ-3 (отпуск с ЦС)"
exp['Движение требует МХ-3 (отпуск с ЦС)'] = exp['Сцепка МХ-3'].map(counts_).fillna(0).astype(int)

# получаем столбец "М-11"
exp['М-11'] = np.where(
    (exp['Вид движения'].isin([301, 311, 351])) &
    ((exp['Вид докум. закупки'] == 'ZUB') | (exp['Вид докум. закупки'] == '')) &
    (exp['Отпускающий завод'] != '4072'),
    (exp['Отпускающий завод'] + exp['Отпускающий склад']).map(counts).fillna(0).astype(int),
    0
)

# получаем столбец "М-15"
exp['М-15'] = np.where(
    (exp['Вид движения'].isin([301, 311, 351])) &
    (exp['Вид докум. закупки'] == 'ZUP') &
    (exp['Отпускающий завод'] != '4072'),
    (exp['Отпускающий завод'] + exp['Отпускающий склад']).map(counts).fillna(0).astype(int),
    0
)

# получаем столбец "МБ-7"
exp['МБ-7'] = np.where(
    (exp['Вид движения'] == 941) &
    (exp['Отпускающий завод'] != '4072') &
    (~exp['Класс оценки'].isin(['0800', '0802'])),
    (exp['Отпускающий завод'] + exp['Отпускающий склад']).map(counts).fillna(0).astype(int),
    0
)

# получаем столбец "АСП"
exp['АСП'] = np.where(
    (exp['Вид движения'].isin([221, 291, 991])) &
    (exp['Отпускающий завод'] == '4072'),
    1,
    0
)

# получаем столбец "Документ материала ГПН-Снаб (из выгрузки зеркл для МХ3)"
exp = (exp
       .merge(
           mirr_mx3[['Документ материала ДЗО', 'Документ материала ГПН-Снаб']],
           left_on='Документ материала',
           right_on='Документ материала ДЗО',
           how='left'
           )
       .drop('Документ материала ДЗО', axis=1)
)
exp['Документ материала ГПН-Снаб'] = exp['Документ материала ГПН-Снаб'].astype('Int64').fillna(0).astype(str)
exp.loc[exp['Документ материала ГПН-Снаб'] == '0', 'Документ материала ГПН-Снаб'] = ''
exp = exp.rename(columns={'Документ материала ГПН-Снаб':'Документ материала ГПН-Снаб (из выгрузки зеркл для МХ3)'})

# получаем столбец "Дивизион"
exp = (exp
       .assign(key=exp['Отпускающий завод'] + exp['Отпускающий склад'])
       .merge(manual[['сцепка на мх1/м11', 'Дивизион.1']], left_on='key', right_on='сцепка на мх1/м11', how='left')
       .drop(['key', 'сцепка на мх1/м11'], axis=1)
       .rename(columns={'Дивизион.1':'Дивизион'})
)
exp['Дивизион'] = exp['Дивизион'].fillna('')

# получаем столбец "ИТОГ в свод по М-15"
exp['ИТОГ в свод по М-15'] = np.where(
    (exp['Отпускающий завод'] + exp['Отпускающий склад']) == (exp['Принимающий завод'] + exp['Принимающий склад']), 0, 1
    ) * exp['М-15']

# получаем столбец "МХ-3"
exp['МХ-3'] = np.where(
    exp['Документ материала ГПН-Снаб (из выгрузки зеркл для МХ3)'].isna(), 0, 1
    ) * exp['Движение требует МХ-3 (отпуск с ЦС)']

# получаем столбец "НЭД для МХ3 (из выгрузки журнала расхода после фильтрации ГПН-Снаб)"
exp = (exp
       .merge(
           zinmm521_mirr_mx3[['Документ материала', 'НЭД']],
           left_on='Документ материала ГПН-Снаб (из выгрузки зеркл для МХ3)',
           right_on='Документ материала',
           how='left')
       .drop('Документ материала_y', axis=1)
       .rename(columns={
           'Документ материала_x':'Документ материала',
           'НЭД_x':'НЭД',
           'НЭД_y':'НЭД для МХ3 (из выгрузки журнала расхода после фильтрации ГПН-Снаб)'
           })
)
exp['НЭД для МХ3 (из выгрузки журнала расхода после фильтрации ГПН-Снаб)'] = exp['НЭД для МХ3 (из выгрузки журнала расхода после фильтрации ГПН-Снаб)'].fillna('')

# получаем столбец "Электронный ПУД МХ3"
exp['Электронный ПУД МХ3'] = np.where(exp['НЭД для МХ3 (из выгрузки журнала расхода после фильтрации ГПН-Снаб)']=='', 0, 1) * exp['МХ-3']

# получаем столбец "Электронный ПУД для АСП"
exp['Электронный ПУД для АСП'] = np.where(exp['НЭД']=='', 0, 1) * exp['АСП']

# получаем столбец "Электронный ПУД для МБ-7"
exp['Электронный ПУД для МБ-7'] = np.where(exp['НЭД']=='', 0, 1) * exp['МБ-7']

# получаем столбец "Электронный ПУД для М-11"
exp['Электронный ПУД для М-11'] = np.where(exp['НЭД']=='', 0, 1) * exp['М-11']

# получаем столбец "ИТОГ в свод по М-11"
exp['ИТОГ в свод по М-11'] = np.where(
    (exp['Отпускающий завод'] + exp['Отпускающий склад']) == (exp['Принимающий завод'] + exp['Принимающий склад']), 0, 1
    ) * exp['М-11']

## Свод

In [42]:
# для всех заводов фиксируем формат - четыре цифры, по необходимости дополнеяем ведущими нулями
final['Завод ДО'] = final['Завод ДО'].astype(str).str.zfill(4).str.upper()

In [43]:
# подсчет МХ-1
res_mx1 = (final
           .merge(
               inc[['Принимающий завод', 'Дивизион', 'Электронный ПУД МХ1']],
               left_on=['Завод ДО', 'Дивизион'],
               right_on=['Принимающий завод','Дивизион'],
               how='left'
               )
           .groupby(['Дочернее общество', 'Завод ДО', 'Дивизион'], sort=False)['Электронный ПУД МХ1']
           .sum()
           .reset_index()
           .rename(columns={'Электронный ПУД МХ1':'МХ-1'})
)

# подсчет ВСЕГО МХ-1
res_mx1_all = (final
               .merge(
                   inc[['Принимающий завод', 'Дивизион', 'МХ1']],
                   left_on=['Завод ДО', 'Дивизион'],
                   right_on=['Принимающий завод','Дивизион'],
                   how='left'
                   )
               .groupby(['Дочернее общество', 'Завод ДО', 'Дивизион'], sort=False)['МХ1']
               .sum()
               .reset_index()
               .rename(columns={'МХ1':'ВСЕГО МХ-1'})
)

# подсчет М4
res_m4 = (final
          .merge(
              inc[['Принимающий завод', 'Дивизион', 'Электронный ПУД М4']],
              left_on=['Завод ДО', 'Дивизион'],
              right_on=['Принимающий завод','Дивизион'],
              how='left'
              )
          .groupby(['Дочернее общество', 'Завод ДО', 'Дивизион'], sort=False)['Электронный ПУД М4']
          .sum()
          .reset_index()
          .rename(columns={'Электронный ПУД М4':'М4'})
)

# подсчет ВСЕГО М4
res_m4_all = (final
              .merge(
                  inc[['Принимающий завод', 'Дивизион', 'Количество М4']],
                  left_on=['Завод ДО', 'Дивизион'],
                  right_on=['Принимающий завод','Дивизион'],
                  how='left'
                  )
              .groupby(['Дочернее общество', 'Завод ДО', 'Дивизион'], sort=False)['Количество М4']
              .sum()
              .reset_index()
              .rename(columns={'Количество М4':'ВСЕГО М4'})
)

# подсчет МХ3
res_mx3 = (final
           .merge(
               exp[['Отпускающий завод', 'Дивизион', 'Электронный ПУД МХ3']],
               left_on=['Завод ДО', 'Дивизион'],
               right_on=['Отпускающий завод','Дивизион'],
               how='left'
               )
           .groupby(['Дочернее общество', 'Завод ДО', 'Дивизион'], sort=False)['Электронный ПУД МХ3']
           .sum()
           .reset_index()
           .rename(columns={'Электронный ПУД МХ3':'МХ3'})
)

# подсчет ВСЕГО МХ3
res_mx3_all = (final
               .merge(
                   exp[['Отпускающий завод', 'Дивизион', 'МХ-3']],
                   left_on=['Завод ДО', 'Дивизион'],
                   right_on=['Отпускающий завод','Дивизион'],
                   how='left'
                   )
               .groupby(['Дочернее общество', 'Завод ДО', 'Дивизион'], sort=False)['МХ-3']
               .sum()
               .reset_index()
               .rename(columns={'МХ-3':'ВСЕГО МХ3'})
)

# подсчет М-11
res_m11 = (final
           .merge(
               exp[['Отпускающий завод', 'Дивизион', 'Электронный ПУД для М-11']],
               left_on=['Завод ДО', 'Дивизион'],
               right_on=['Отпускающий завод','Дивизион'],
               how='left'
               )
           .groupby(['Дочернее общество', 'Завод ДО', 'Дивизион'], sort=False)['Электронный ПУД для М-11']
           .sum()
           .reset_index()
           .rename(columns={'Электронный ПУД для М-11':'М-11'})
)

# подсчет ВСЕГО М-11
res_m11_all = (final
               .merge(
                   exp[['Отпускающий завод', 'Дивизион', 'ИТОГ в свод по М-11']],
                   left_on=['Завод ДО', 'Дивизион'],
                   right_on=['Отпускающий завод','Дивизион'],
                   how='left'
                   )
               .groupby(['Дочернее общество', 'Завод ДО', 'Дивизион'], sort=False)['ИТОГ в свод по М-11']
               .sum()
               .reset_index()
               .rename(columns={'ИТОГ в свод по М-11':'ВСЕГО М-11'})
)

# подсчет МБ7
res_mb7 = (final
           .merge(
               exp[['Отпускающий завод', 'Дивизион', 'Электронный ПУД для МБ-7']],
               left_on=['Завод ДО', 'Дивизион'],
               right_on=['Отпускающий завод','Дивизион'],
               how='left'
               )
           .groupby(['Дочернее общество', 'Завод ДО', 'Дивизион'], sort=False)['Электронный ПУД для МБ-7']
           .sum()
           .reset_index()
           .rename(columns={'Электронный ПУД для МБ-7':'МБ7'})
)

# подсчет ВСЕГО МБ7
res_mb7_all = (final
               .merge(
                   exp[['Отпускающий завод', 'Дивизион', 'МБ-7']],
                   left_on=['Завод ДО', 'Дивизион'],
                   right_on=['Отпускающий завод','Дивизион'],
                   how='left'
                   )
               .groupby(['Дочернее общество', 'Завод ДО', 'Дивизион'], sort=False)['МБ-7']
               .sum()
               .reset_index()
               .rename(columns={'МБ-7':'ВСЕГО МБ7'})
)

# подсчет АСП
res_asp = (final
           .merge(
               exp[['Отпускающий завод', 'Дивизион', 'Электронный ПУД для АСП']],
               left_on=['Завод ДО', 'Дивизион'],
               right_on=['Отпускающий завод','Дивизион'],
               how='left'
               )
           .groupby(['Дочернее общество', 'Завод ДО', 'Дивизион'], sort=False)['Электронный ПУД для АСП']
           .sum()
           .reset_index()
           .rename(columns={'Электронный ПУД для АСП':'АСП'})
)

# подсчет ВСЕГО АСП
res_asp_all = (final
               .merge(
                   exp[['Отпускающий завод', 'Дивизион', 'АСП']],
                   left_on=['Завод ДО', 'Дивизион'],
                   right_on=['Отпускающий завод','Дивизион'],
                   how='left'
                   )
               .groupby(['Дочернее общество', 'Завод ДО', 'Дивизион'], sort=False)['АСП']
               .sum()
               .reset_index()
               .rename(columns={'АСП':'ВСЕГО АСП'})
)

In [44]:
# присоединяем к своду отдельные таблицы с подсчетами
final = (final
         .merge(res_mx1, on=['Дочернее общество', 'Завод ДО', 'Дивизион'], how='left')
         .merge(res_mx1_all, on=['Дочернее общество', 'Завод ДО', 'Дивизион'], how='left')
         .merge(res_m4, on=['Дочернее общество', 'Завод ДО', 'Дивизион'], how='left')
         .merge(res_m4_all, on=['Дочернее общество', 'Завод ДО', 'Дивизион'], how='left')
         .merge(res_mx3, on=['Дочернее общество', 'Завод ДО', 'Дивизион'], how='left')
         .merge(res_mx3_all, on=['Дочернее общество', 'Завод ДО', 'Дивизион'], how='left')
         .merge(res_m11, on=['Дочернее общество', 'Завод ДО', 'Дивизион'], how='left')
         .merge(res_m11_all, on=['Дочернее общество', 'Завод ДО', 'Дивизион'], how='left')
         .merge(res_mb7, on=['Дочернее общество', 'Завод ДО', 'Дивизион'], how='left')
         .merge(res_mb7_all, on=['Дочернее общество', 'Завод ДО', 'Дивизион'], how='left')
         .merge(res_asp, on=['Дочернее общество', 'Завод ДО', 'Дивизион'], how='left')
         .merge(res_asp_all, on=['Дочернее общество', 'Завод ДО', 'Дивизион'], how='left')
)

# меняем дробный тип данных на целочисленный
for col in final.columns[3:]:
  final[col] = pd.to_numeric(final[col], errors='coerce').astype('Int64')

# добавляем подсчет процентов
final['% МХ-1'] = (round(final['МХ-1'] / final['ВСЕГО МХ-1'] * 100, 1).astype(str) + '%').str.replace('nan%', '')
final['% М4'] = (round(final['М4'] / final['ВСЕГО М4'] * 100, 1).astype(str) + '%').str.replace('nan%', '')
final['% МХ3'] = (round(final['МХ3'] / final['ВСЕГО МХ3'] * 100, 1).astype(str) + '%').str.replace('nan%', '')
final['% М-11'] = (round(final['М-11'] / final['ВСЕГО М-11'] * 100, 1).astype(str) + '%').str.replace('nan%', '')
final['% МБ7'] = (round(final['МБ7'] / final['ВСЕГО МБ7'] * 100, 1).astype(str) + '%').str.replace('nan%', '')
final['% АСП'] = (round(final['АСП'] / final['ВСЕГО АСП'] * 100, 1).astype(str) + '%').str.replace('nan%', '')

# пересортировка
final = final[[
    'Дочернее общество', 'Завод ДО', 'Дивизион',
    'МХ-1', 'ВСЕГО МХ-1', '% МХ-1',
    'М4', 'ВСЕГО М4', '% М4',
    'МХ3', 'ВСЕГО МХ3', '% МХ3',
    'М-11', 'ВСЕГО М-11', '% М-11',
    'МБ7', 'ВСЕГО МБ7', '% МБ7',
    'АСП', 'ВСЕГО АСП', '% АСП',
    ]]

# формируем итоговую строку
total_row = {
    'Дочернее общество': '',
    'Завод ДО': '',
    'Дивизион': '',
    'МХ-1': final['МХ-1'].sum(),
    'ВСЕГО МХ-1': final['ВСЕГО МХ-1'].sum(),
    '% МХ-1': round(final['МХ-1'].sum() / final['ВСЕГО МХ-1'].sum() * 100, 0).astype(int).astype(str)+'%',
    'М4': final['М4'].sum(),
    'ВСЕГО М4': final['ВСЕГО М4'].sum(),
    '% М4': round(final['М4'].sum() / final['ВСЕГО М4'].sum() * 100, 0).astype(int).astype(str)+'%',
    'МХ3': final['МХ3'].sum(),
    'ВСЕГО МХ3': final['ВСЕГО МХ3'].sum(),
    '% МХ3': round(final['МХ3'].sum() / final['ВСЕГО МХ3'].sum() * 100, 0).astype(int).astype(str)+'%',
    'М-11': final['М-11'].sum(),
    'ВСЕГО М-11': final['ВСЕГО М-11'].sum(),
    '% М-11': round(final['М-11'].sum() / final['ВСЕГО М-11'].sum() * 100, 0).astype(int).astype(str)+'%',
    'МБ7': final['МБ7'].sum(),
    'ВСЕГО МБ7': final['ВСЕГО МБ7'].sum(),
    '% МБ7': round(final['МБ7'].sum() / final['ВСЕГО МБ7'].sum() * 100, 0).astype(int).astype(str)+'%',
    'АСП': final['АСП'].sum(),
    'ВСЕГО АСП': final['ВСЕГО АСП'].sum(),
    '% АСП': round(final['АСП'].sum() / final['ВСЕГО АСП'].sum() * 100, 0).astype(int).astype(str)+'%'
}

# добавляем итоговую строку
final.loc[len(final)] = total_row

final

,Дочернее общество,Завод ДО,Дивизион,МХ-1,ВСЕГО МХ-1,% МХ-1,М4,ВСЕГО М4,% М4,МХ3,...,% МХ3,М-11,ВСЕГО М-11,% М-11,МБ7,ВСЕГО МБ7,% МБ7,АСП,ВСЕГО АСП,% АСП
0,"АО ""Мессояханефтегаз""",5061,Север,75,75,100.0%,79,79,100.0%,215,...,98.6%,201,201,100.0%,60,60,100.0%,0,0,
1,"ООО ""Меретояханефтегаз""",1061,Север,36,36,100.0%,39,39,100.0%,2,...,6.5%,21,40,52.5%,29,29,100.0%,0,0,
2,"ООО ""ГПН-Ямал""",1241,Север,35,36,97.2%,42,43,97.7%,12,...,18.2%,162,162,100.0%,61,61,100.0%,0,0,
3,"ООО ""ГПН-Развитие""",0801,Север,0,0,,0,0,,0,...,,0,0,,0,0,,0,0,
4,"ООО ""ГПН-Развитие""",0811,Север,0,0,,0,0,,0,...,,0,0,,0,0,,0,0,
5,"ООО ""ГПН-Развитие""",0821,Север,48,48,100.0%,48,48,100.0%,0,...,,6,6,100.0%,0,0,,0,0,
6,"ООО ""ГПН-Развитие""",0501,Север,0,0,,0,0,,0,...,,2,2,100.0%,0,0,,0,0,
7,"ООО ""ГПН-Развитие""",0601,Север,0,0,,0,0,,0,...,,0,0,,0,0,,0,0,
8,"ООО ""ГПН-ГЕО""",6791,Север,2,2,100.0%,2,2,100.0%,0,...,0.0%,1,5,20.0%,0,0,,0,0,
9,"ООО ""ГПН-Оренбург""",1261,Север,53,53,100.0%,54,54,100.0%,102,...,36.8%,148,157,94.3%,173,173,100.0%,0,0,


In [45]:
# краткие результаты
res_briefly = pd.DataFrame({
    '': ['ЭДО', 'Общее количество', ''],
    'ВСЕГО': [
        total_row['МХ-1'] + total_row['М4'] + total_row['МХ3'] + total_row['М-11'] + total_row['МБ7'] + total_row['АСП'],
        total_row['ВСЕГО МХ-1'] + total_row['ВСЕГО М4'] + total_row['ВСЕГО МХ3'] + total_row['ВСЕГО М-11'] + total_row['ВСЕГО МБ7'] + total_row['ВСЕГО АСП'],
        0]
})

# расчет процента
res_briefly.loc[2, 'ВСЕГО'] = (
    round(res_briefly.loc[0, 'ВСЕГО'] / res_briefly.loc[1, 'ВСЕГО'] * 100, 0).astype(int).astype(str)+'%'
    if res_briefly.loc[1, 'ВСЕГО'] != 0 else '0%'
)

res_briefly

/tmp/ipython-input-2913030917.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '87%' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  res_briefly.loc[2, 'ВСЕГО'] = (


,,ВСЕГО
0,ЭДО,8242
1,Общее количество,9492
2,,87%


In [46]:
# для расчета итоговой таблицы добавляем два побочных поля (потом удалим)
final['ЭДО'] = final['МХ-1'] + final['М4'] + final['МХ3'] + final['М-11'] + final['МБ7'] + final['АСП']
final['ВСЕГО'] = final['ВСЕГО МХ-1'] + final['ВСЕГО М4'] + final['ВСЕГО МХ3'] + final['ВСЕГО М-11'] + final['ВСЕГО МБ7'] + final['ВСЕГО АСП']

# агрегация по дивизионам
result = final.groupby('Дивизион')[['ЭДО', 'ВСЕГО']].sum().reset_index()

# исключаем сумму по дивизионам
result = result[result['Дивизион'] != '']

# считаем проценты
result['%'] = round(result['ЭДО'] / result['ВСЕГО'] * 100, 0).astype(int).astype(str)+'%'

# удаляем побочные поля
final = final.drop(['ЭДО', 'ВСЕГО'], axis=1)

# добавляем мультииндекс для столбцов
# result.columns = pd.MultiIndex.from_tuples([
#     ('Документы', 'Дивизион'),
#     ('Документы', 'ЭДО'),
#     ('Документы', 'ВСЕГО'),
#     ('Документы', '%')
# ])

result

,Дивизион,ЭДО,ВСЕГО,%
1,Восток,583,707,82%
2,Восточная Сибирь,763,833,92%
3,Запад,107,113,95%
4,Север,4264,5059,84%
5,Северо-запад,6,8,75%
6,Центр,2464,2679,92%
7,Юг,55,93,59%


# Сохранение результатов в файл Excel

In [47]:
# данные для листов с несколькими таблицами
datasets_multiple = {'СВОД': [
    final,
    res_briefly,
    result
    ]}

# данные для листов с одной таблицей
datasets_single = {
    'Приход': inc,
    'Зерк мх1': mirr_mx1,
    '521 после зерк мх1': zinmm521_mirr_mx1,
    'Расход': exp,
    'Зерк мх3': mirr_mx3,
    '521 после зерк мх3': zinmm521_mirr_mx3,
    'Справочник': manual
    }

# наименование отчета
excel_filename = 'Отчет ЭДО.xlsx'

# создаем объект для записи в Excel
with pd.ExcelWriter(excel_filename) as writer:

  # обработка листов с несколькими таблицами
  for sheet_name, tables_list in datasets_multiple.items():
    start_row = 0
    for table_data in tables_list:
      df = pd.DataFrame(table_data)

      # размещаем каждую таблицу последовательно сверху вниз
      df.to_excel(writer, sheet_name=sheet_name, startrow=start_row, startcol=0, header=True, index=False)

      # определяем новую позицию для следующей таблицы - длина таблицы + 2 пустых ряда вниз
      start_row += len(df) + 2

  # обработка листов с одиночными таблицами
  for sheet_name, data in datasets_single.items():
    df = pd.DataFrame(data)
    df.to_excel(writer, sheet_name=sheet_name, header=True, index=False)